# Logger Wrapper personalization/configuration

We will see on this notebook how to personalize the logger wrapper defined by sinergym.

In [7]:
import gym
import numpy as np
import sinergym
from sinergym.utils.wrappers import (LoggerWrapper, MultiObsWrapper,
                                     NormalizeObservation)
from sinergym.utils.constants import RANGES_5ZONE

## Step 1 Inherit and modify the CSVloger

First we need to change the CSV logger to modify the values writen into the file on the funtion create_row_contents

In [ ]:

from sinergym.utils.logger import CSVLogger
from typing import Any, Dict, Optional, Sequence, Tuple, Union, List

class CustomCSVLogger(CSVLogger):

    def __init__(
            self,
            monitor_header: str,
            progress_header: str,
            log_progress_file: str,
            log_file: Optional[str] = None,
            flag: bool = True):
        super(CustomCSVLogger, self).__init__(monitor_header,progress_header,log_progress_file,log_file,flag)
        self.last_10_steps_reward = np.zeros(10)

    def create_row_contents(
            self,
            timestep: int,
            observation: List[Any],
            action: Union[List[Union[int, float]], List[None]],
            simulation_time: float,
            reward: Optional[float],
            total_power_no_units: Optional[float],
            comfort_penalty: Optional[float],
            done: bool) -> List:
        self.last_10_steps_reward.pop(0)
        self.last_10_steps_reward.append(reward)

        return [timestep] + list(observation) + \
                       list(action) + [simulation_time, reward,np.mean(self.last_10_steps_reward),
                                       total_power_no_units, comfort_penalty, done]

## Step 2 Intanciate the LoggerWrapper


now we need to instantiate the loggerwrapper and specify the new headers of our file and the csvlogger class we want to use.

In [9]:
env=gym.make('Eplus-demo-v1')
env=LoggerWrapper(env,logger=CustomCSVLogger,headers = ['timestep'] + env.variables['observation'] +
                env.variables['action'] + ['time (seconds)', 'reward', '10-mean-reward',
                'power_penalty', 'comfort_penalty', 'done'])

[2022-05-23 16:42:01,844] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Updating idf Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2022-05-23 16:42:01,844] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Updating idf Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2022-05-23 16:42:01,844] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Updating idf Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2022-05-23 16:42:01,844] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Updating idf Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2022-05-23 16:42:01,844] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Updating idf Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2022-05-23 16:42:01,844] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Updating idf Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2022-05-23 16:42:01,847] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Updating idf OutPut:Variable and variable

Now, you can see in Sinergym output folder that you will have available `progress.csv` file and `monitor.csv` files in each episode.

In [12]:
for i in range(1):
    obs = env.reset()
    rewards = []
    done = False
    current_month = 0
    while not done:
        a = env.action_space.sample()
        obs, reward, done, info = env.step(a)
        rewards.append(reward)
        if info['month'] != current_month:  # display results every month
            current_month = info['month']
            print('Reward: ', sum(rewards), info)
    print('Episode ', i, 'Mean reward: ', np.mean(
        rewards), 'Cumulative reward: ', sum(rewards))
env.close()

[2022-05-23 16:42:10,692] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2022-05-23 16:42:10,692] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2022-05-23 16:42:10,692] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2022-05-23 16:42:10,692] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2022-05-23 16:42:10,692] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2022-05-23 16:42:10,692] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2022-05-23 16:42:10,692] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2022-05-23 16:42:10,692] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2022-05-23 16:42:10,709] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/

Reward:  -0.3808358083250143 {'timestep': 1, 'time_elapsed': 900, 'year': 1991, 'month': 1, 'day': 1, 'hour': 0, 'total_power': 7616.716166500285, 'total_power_no_units': -0.7616716166500286, 'comfort_penalty': -0.0, 'temperatures': [20.99998783039325], 'out_temperature': 1.8, 'action_': [21, 21]}
Reward:  -1871.2735943472342 {'timestep': 2976, 'time_elapsed': 2678400, 'year': 1991, 'month': 2, 'day': 1, 'hour': 0, 'total_power': 4717.088000520113, 'total_power_no_units': -0.4717088000520114, 'comfort_penalty': -1.7573715796282414, 'temperatures': [18.24262842037176], 'out_temperature': -7.0, 'action_': [17, 28]}
Reward:  -3591.708406946749 {'timestep': 5664, 'time_elapsed': 5097600, 'year': 1991, 'month': 3, 'day': 1, 'hour': 0, 'total_power': 1914.018114977942, 'total_power_no_units': -0.19140181149779423, 'comfort_penalty': -1.0756168928023513, 'temperatures': [18.92438310719765], 'out_temperature': 8.1, 'action_': [19, 26]}
Reward:  -4739.976056551516 {'timestep': 8640, 'time_elaps

[2022-05-23 16:42:33,727] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:EnergyPlus simulation closed successfully. 
[2022-05-23 16:42:33,727] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:EnergyPlus simulation closed successfully. 
[2022-05-23 16:42:33,727] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:EnergyPlus simulation closed successfully. 
[2022-05-23 16:42:33,727] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:EnergyPlus simulation closed successfully. 
[2022-05-23 16:42:33,727] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:EnergyPlus simulation closed successfully. 
[2022-05-23 16:42:33,727] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:EnergyPlus simulation closed successfully. 
[2022-05-23 16:42:33,727] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:EnergyPlus simulation closed successfully. 
[2022-05-23 16:42:33,727] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:EnergyPlus simulation closed successfully. 
